<a href="https://colab.research.google.com/github/sudhakarmlal/EIPPhase2/blob/master/Assignment6EIP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 6 (EIP Phase2 : Session2)
##Project LSTM


###Description:

####The project is  about generating 500 texts using LSTM Model.
#### The LSTM  Model to be trained on a dataset from Project GutenBerg.The Dataset(Book) on which the LSTM Model to be trained on  is 'WonderLand.txt'


#### All the punctuations to be removed from  the source text

#### To train the model padded sequences to be used instead of random sequences of characters.


### The Approach

#### Step1: Read the file as raw text from wonderland.txt.Mounted in the google drive.

####Step2:Remove the punctuations from the files

####Step3:Convert all the unique characters used in the text to int.Also find the vocab count which is the unique characters used 


#### Step4: Split the source text by "\n"  while splitting the source text check the max length of the splits produced.

####Step5:Pad each split produced with the max length derived from the previoius step


####Step6:Merge these padded sequences of splits take this as the clean text from which train data should be generated


####Step7: Generate train data(X_train) by sliding  10 characters each time.Take sequence length as max length of the splits produced dervied in step4.


####:Step8:Feed this Data to LSTM Model.


####:Step9:One the Model is trained.Generate 500 character sequences by providing an input  of random index from the train data(X_train)


 





### Import necessary Libraries

In [2]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


### Mount the groogle drive in google colab to read the wonderland file(to be used by LSTM Model)

In [3]:

from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


### Load the file wonderland.txt and read the text in the file

In [0]:
filename = "/content/drive/My Drive/SequenceModels/LSTM/wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

#### Check the length of raw text

In [5]:
print(len(raw_text))

144437


####Check the unique characters used in the source text and also convert these characters to int
####Note:integral values to be fed to LSTM model and not characters.Hence the characters to be converted to int

In [0]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))


In [7]:
print(chars)

['\n', ' ', '!', '"', "'", '(', ')', '*', ',', '-', '.', '0', '3', ':', ';', '?', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [8]:
print(char_to_int)

{'\n': 0, ' ': 1, '!': 2, '"': 3, "'": 4, '(': 5, ')': 6, '*': 7, ',': 8, '-': 9, '.': 10, '0': 11, '3': 12, ':': 13, ';': 14, '?': 15, '[': 16, ']': 17, '_': 18, 'a': 19, 'b': 20, 'c': 21, 'd': 22, 'e': 23, 'f': 24, 'g': 25, 'h': 26, 'i': 27, 'j': 28, 'k': 29, 'l': 30, 'm': 31, 'n': 32, 'o': 33, 'p': 34, 'q': 35, 'r': 36, 's': 37, 't': 38, 'u': 39, 'v': 40, 'w': 41, 'x': 42, 'y': 43, 'z': 44, '\ufeff': 45}


In [9]:
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  144437
Total Vocab:  46


#### Remove punctuations from the source text

In [0]:
punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

In [0]:
no_punct_text = ""
for char in raw_text:
   if char not in punctuations:
       no_punct_text = no_punct_text + char

In [13]:
print(len(no_punct_text))

136117


In [0]:
no_punct_text=no_punct_text.replace("\n\n","\n")

In [15]:
len(no_punct_text)

135284

#### Check the  unique characters and corresponding int values after removal of punctuations

In [0]:
chars = sorted(list(set(no_punct_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [17]:
print(chars)

['\n', ' ', '0', '3', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [18]:
print(char_to_int)

{'\n': 0, ' ': 1, '0': 2, '3': 3, 'a': 4, 'b': 5, 'c': 6, 'd': 7, 'e': 8, 'f': 9, 'g': 10, 'h': 11, 'i': 12, 'j': 13, 'k': 14, 'l': 15, 'm': 16, 'n': 17, 'o': 18, 'p': 19, 'q': 20, 'r': 21, 's': 22, 't': 23, 'u': 24, 'v': 25, 'w': 26, 'x': 27, 'y': 28, 'z': 29, '\ufeff': 30}


#### Print total characters in the text and total unique characters after removal of punctuation

In [19]:
n_chars = len(no_punct_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  135284
Total Vocab:  31


#### Split the source text by "\n" to get the corpus

In [0]:
corpus = no_punct_text.lower().split("\n") 

In [21]:
len(corpus)

2512

#### Further remove empty lines from the source text and make the final corpus.While making the final corpus just check the max length of all the splits produced to be used for padding

In [22]:
max = max=len(corpus[0])
#len(corpus)
ctr =0
final_corpus = []
#final_corpus = [0 for i in range(len(corpus))]
for i in range(len(corpus)):
  #print(len(corpus[i]))
  if len(corpus[i]) != 0:
    final_corpus.append(corpus[i])
  if len(corpus[i]) >= max:
    max=len(corpus[i])  
  

print(max)
print(len(final_corpus))

73
2481


#### Pad each split by the max length derived in the previous step

In [0]:
padded_corpus =[]
for i in range(len(final_corpus)):
  pad=73-len(final_corpus[i])
  #print("Pad",pad)
  #print(final_corpus[i])
  padded =str(final_corpus[i]).ljust(73," ")
  #print("afterpadding",len(padded))
  padded_corpus.append(padded)
  

#### Verify if padding is proper by printing the length of 10 elements of padded corpus

In [24]:
for i in range(10):
  print(len(padded_corpus[i]))

73
73
73
73
73
73
73
73
73
73


#### Merge the Padded sequence to create a clean text which is the  free from punctionations,empty lines and is padded(padded after the split on "\n")

In [0]:
clean_text=""
for i in range(len(padded_corpus)):
  clean_text = clean_text+padded_corpus[i]

In [0]:
chars = sorted(list(set(clean_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [27]:
n_chars = len(clean_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  181113
Total Vocab:  30


In [0]:
dataX = []
dataY = []
for i in range(len(padded_corpus)-1):
  seq_in=padded_corpus[i]
  seq_out=padded_corpus[i+1][0]
  dataX.append([char_to_int[char] for char in seq_in])
  dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
  

#### Form train data from the clean text produced in the previous step.
#### While training take a window slide of 10 characters and sequence length of  73 which is max length of the splits(derived in the previous step) 

In [30]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 73
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 10):
	seq_in = clean_text[i:i + seq_length]
	seq_out = clean_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])

#cntr = 0
#for i in range(0, n_chars - seq_length, 1):
  #print("Hi")
  #if cntr < 20:
    #print(raw_text[i:i + seq_length])
    #print(len(raw_text[i:i + seq_length]))
    #print(raw_text[i + seq_length])
  #cntr = cntr +1
#print(cntr)
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  18104


#### Reshape the train data for feeding to LSTM

In [0]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, 73, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

#### Build the LSTM model

In [32]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True,dropout=0.1))
model.add(Dropout(0.1))
model.add(LSTM(256))
#model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

W0727 10:42:05.182772 140188924004224 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0727 10:42:05.219854 140188924004224 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0727 10:42:05.373200 140188924004224 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0727 10:42:05.627557 140188924004224 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0727 10:42:05.638804 

#### Define the check point to save the best Model

In [0]:

# define the checkpoint
#filepath="/content/drive/My Drive/EIPSession2/SequenceModels/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
filepath="/content/drive/My Drive/SequenceModels/LSTM/weights-improv.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]


#### Run the Model (Batch Size 64 and number of epochs 100)

In [34]:
#model.fit(X, y, epochs=5, batch_size=128, callbacks=callbacks_list)
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X, y, epochs=100, batch_size=64, callbacks=callbacks_list)

W0727 10:42:24.854573 140188924004224 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0727 10:42:24.884869 140188924004224 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W0727 10:42:25.035172 140188924004224 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
18104/18104 [==============================] - 60s 3ms/step - loss: 2.3520

Epoch 00001: loss improved from inf to 2.35201, saving model to /content/drive/My Drive/SequenceModels/LSTM/weights-improv.hdf5
Epoch 2/100
18104/18104 [==============================] - 58s 3ms/step - loss: 2.2473

Epoch 00002: loss improved from 2.35201 to 2.24733, saving model to /content/drive/My Drive/SequenceModels/LSTM/weights-improv.hdf5
Epoch 3/100
18104/18104 [==============================] - 58s 3ms/step - loss: 2.1593

Epoch 00003: loss improved from 2.24733 to 2.15935, saving model to /content/drive/My Drive/SequenceModels/LSTM/weights-improv.hdf5
Epoch 4/100
18104/18104 [==============================] - 57s 3ms/step - loss: 2.0464

Epoch 00004: loss improved from 2.15935 to 2.04641, saving model to /content/drive/My Drive/SequenceModels/LSTM/weights-improv.hdf5
Epoch 5/100
18104/18104 [==============================] - 57s 3ms/step - loss: 1.9776

Epoch 00005: loss improved from 2.04

#### Once Model is trained load the best weights of the Model

In [0]:
# load the network weights
filename = "/content/drive/My Drive/EIPSession2/SequenceModels/weights-improvement.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

####  Convert int to characters to be used to generate text of 500 character sequences

In [0]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

#### Predict 500 character sequences:

####  Generate 500 character sequences based on the training data input fed to the model.
#### A random seed of training data is fed to the Model as input for predicting 500 character sequences

In [36]:
start = numpy.random.randint(0, len(dataX)-1)
print("Start is",start)
pattern = dataX[start]

print("Pattern is",pattern)
print("Pattern length is",len(pattern))
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(500):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	print(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

Start is 6773
Pattern is [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 23, 22, 0, 11, 15, 0, 16, 17, 22, 0, 3, 0, 21, 7, 20, 18, 7, 16, 22, 0, 11, 0, 22, 7, 14, 14, 0, 27, 17, 23, 0, 21, 3, 11, 6, 0, 3, 14, 11, 5, 7, 0, 11, 15, 0, 3, 11, 15, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0]
Pattern length is 73
Seed:
"               but im not a serpent i tell you said alice im aim a         "
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
a
n
i
c
e
 
t
a
n
d
 
n
n
 
t
h
e
 
d
a
k
e
e
 
h
v
 
a
l
l
c
e
 
f
a
n
l
 
b
n
d
 
t
a
s
 
a
 
b
u
w
i
 
t
n
t
n
 
h
t
i
n
g
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
t
h
e
 
q
i
g
l
t
 
i
i
a
d
 
i
o
r
s
t
e
d
 
a
 
b
o
d
 
w
h
e
n
e
 
w
h
e
 
l
a
d
 
b
u
c
h
e
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
a
 
b
i
i
 
t
h
e
y
 
t
a
i
d
 
t
h
e
 
h
a
t
t
e
r
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 